In [1]:
import pandas as pd
import os
from sodapy import Socrata

# cvs_output_file_path =  os.path.join("cvs_output.csv")

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cdc.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cdc.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("8pt5-q6wp", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,indicator,group,state,subgroup,week,week_label,value,lowci,highci,confidence_interval,quartile_range
0,Symptoms of Depressive Disorder,National Estimate,United States,United States,1,Apr 23 - May 5,23.5,22.7,24.3,22.7 - 24.3,NaN
1,Symptoms of Depressive Disorder,By Age,United States,18 - 29 years,1,Apr 23 - May 5,32.7,30.2,35.2,30.2 - 35.2,NaN
2,Symptoms of Depressive Disorder,By Age,United States,30 - 39 years,1,Apr 23 - May 5,25.7,24.1,27.3,24.1 - 27.3,NaN
3,Symptoms of Depressive Disorder,By Age,United States,40 - 49 years,1,Apr 23 - May 5,24.8,23.3,26.2,23.3 - 26.2,NaN
4,Symptoms of Depressive Disorder,By Age,United States,50 - 59 years,1,Apr 23 - May 5,23.2,21.5,25,21.5 - 25.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1995,Symptoms of Anxiety Disorder,By State,Kansas,Kansas,10,July 2 - July 7,32.3,28.2,36.6,28.2 - 36.6,31.4-32.9
1996,Symptoms of Anxiety Disorder,By State,Kentucky,Kentucky,10,July 2 - July 7,36.3,31.2,41.6,31.2 - 41.6,35.4-40.6
1997,Symptoms of Anxiety Disorder,By State,Louisiana,Louisiana,10,July 2 - July 7,38.6,34.2,43.1,34.2 - 43.1,35.4-40.6
1998,Symptoms of Anxiety Disorder,By State,Maine,Maine,10,July 2 - July 7,32.8,29,36.7,29.0 - 36.7,31.4-32.9


In [2]:
kansas_df = results_df.loc[results_df["state"] == "Kansas", :]
kansas_df['indicator'].value_counts()
kansas_df

missouri_df=results_df.loc[results_df['state']=='Missouri']
missouri_df

ks_mo=pd.merge(kansas_df,missouri_df, how='outer')
ks_mo

cleaned_KsMo=ks_mo[['indicator','state','week','value']]
cleaned_KsMo

cleaned_KsMo['value']=cleaned_KsMo['value'].astype(float)

cleaned=cleaned_KsMo.groupby(['week','indicator'])
cleaned.count().head(10)

state_mean=cleaned['value'].mean()
final=pd.DataFrame({'KS & MO Average(%)':state_mean})
final.head()


C:\Users\dhire\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


KS & MO Average(%)
week indicator                                                             
1    Symptoms of Anxiety Disorder                                     28.40
     Symptoms of Anxiety Disorder or Depressive Diso...               33.85
     Symptoms of Depressive Disorder                                  21.55
10   Symptoms of Anxiety Disorder                                     32.30
     Symptoms of Depressive Disorder                                  25.15

In [3]:
all_states=results_df.loc[(results_df['state'] !='United States')]
all_states

cleaned_all_states=all_states[['indicator','state','week','value']]
cleaned_all_states

cleaned_all_states['value']=cleaned_all_states['value'].astype(float)

cleanedAll=cleaned_all_states.groupby(['week','indicator'])
cleanedAll.count().head(10)

country_mean=cleanedAll['value'].mean()
final_total=pd.DataFrame({'National Average(%)':country_mean})
final_total.head()

C:\Users\dhire\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


National Average(%)
week indicator                                                              
1    Symptoms of Anxiety Disorder                                  29.609804
     Symptoms of Anxiety Disorder or Depressive Diso...            34.564706
     Symptoms of Depressive Disorder                               22.247059
10   Symptoms of Anxiety Disorder                                  33.633333
     Symptoms of Depressive Disorder                               26.639216